<a href="https://colab.research.google.com/github/daisysong76/bioinformatics-research/blob/main/lab03b_c146_v02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Science for Biology
#### UC Berkeley PMB/MCB/BioE c146, Fall 2022
# Lab 3b: Continued STR Analysis - Part B v02
##### lab03b-c146-v02.ipynb

For this assignment, we will primarily be using the 'datascience' library.  It is documented here: http://data8.org/datascience/index.html

You are welcome to use other libraries (such as numpy, pandas, etc.) if you prefer

In [ ]:
from datascience import *
%matplotlib inline
import matplotlib.pyplot as plots
import numpy as np
plots.style.use('fivethirtyeight')

### Calculating the Real Match Probabilities
On Tuesday, we started som some exploratory analysis and visualizations of DNA fingerprints. We also did some calculations. These were done ignoring the diploidy of the data. Now that we have a better handle on this dataset, we're going to do the same calculations properly.

#### Background
FBI uses a system called CODIS (Combined DNA Index System) to help support and solve criminal cases. The DNA Profile of a suspect is compared against large DNA databases to look for a match. These DNA databases contain information about people's DNA profiles, which is determined by DNA STR Analysis and consists of one or two alleles at the 20 specific CODIS Core Loci.

The the FBIn FAQ about these is here: https://www.fbi.gov/resources/dna-fingerprint-act-of-2005-expungement-policy/codis-and-ndis-fact-sheet
A description of how these are used is here: https://www.nature.com/scitable/topicpage/forensics-dna-fingerprinting-and-codis-736

The 20 CODIS Core Loci currently being used are:

CSF1PO, FGA, THO1, TPOX, VWA, D3S1358, D5S818, D7S820, D8S1179, D13S317, D16S539, D18S51, D21S11, D1S1656, D2S441, D2S1338, D10S1248, D12S391, D19S433, D22S1045

The 13 Original CODIS Core Loci, which were used before December 31, 2016, were:

CSF1PO, FGA, THO1, TPOX, VWA, D3S1358, D5S818, D7S820, D8S1179, D13S317, D16S539, D18S51, D21S11

Let's import the same dataset as lab03a:

In [ ]:
raw_profiles = Table.read_table("data/CODIS_profiles.csv")

raw_profiles

Group,SampleCode,AMEL,AMEL.1,CSF1PO,CSF1PO.1,D10S1248,D10S1248.1,D12S391,D12S391.1,D13S317,D13S317.1,D16S539,D16S539.1,D18S51,D18S51.1,D19S433,D19S433.1,D1S1656,D1S1656.1,D21S11,D21S11.1,D22S1045,D22S1045.1,D2S1338,D2S1338.1,D2S441,D2S441.1,D3S1358,D3S1358.1,D5S818,D5S818.1,D6S1043,D6S1043.1,D7S820,D7S820.1,D8S1179,D8S1179.1,F13A01,F13A01.1,F13B,F13B.1,FESFPS,FESFPS.1,FGA,FGA.1,LPL,LPL.1,SE33,SE33.1,TH01,TH01.1,TPOX,TPOX.1,vWA,vWA.1,DYS19,DYS389I,DYS389II,DYS390,DYS391,DYS392,DYS393,DYS437,DYS438,DYS439,DYS448,DYS456,DYS458,DYS481,DYS533,DYS549,DYS570,DYS576,DYS635,DYS643,Y-GATA-H4
Cauc,BC11352,X,Y,11,13,13,15,17,18,13,13,8,12,13,15,13,13,14,16.3,28,30,14,16,17,20,14,15,14,17,10,11,12,18,10,12,13,14,5,14,6,9,10,11,24,24,10,12,13,14,6,9.3,8,8,17,17,15,14,30,22,10,11,12,14,9,11,19,15,18,23,12,13,16,19,20,9,12
Hisp,GA05070,X,Y,10,12,12,14,17.3,23,10,11,10,10,16,18,14.2,15,14,16,30,33.2,14,15,19,23,11,15,16,18,11,12,13,13,10,13,10,13,6,7,10,10,11,13,25,26,12,14,13,22,7,9,8,12,14,19,13,14,30,24,10,15,13,14,11,12,20,17,15,25,11,12,16,17,22,11,12
Hisp,GA05071,X,Y,11,15,14,16,19,20,11,12,12,12,17,17,13,13,16,16,30.2,32.2,15,16,17,17,10,10,14,17,11,12,11,13,7,10,13,14,3.2,3.2,8,10,11,12,23,25,9,10,18,26.2,7,9.3,8,11,16,17,14,13,30,24,10,13,13,15,12,13,19,15,16,22,12,13,18,18,23,10,12
Cauc,GC03394,X,Y,10,12,15,16,19,19,11,11,12,13,15,17,14,15,12,15,30,31,15,16,17,20,11,12,15,18,11,12,13,19,11,12,12,12,5,6,6,9,11,11,21,21,9,10,19,22.2,6,7,8,8,17,18,15,12,28,24,10,11,12,15,9,12,19,13,16,22,12,13,18,16,21,9,11
Cauc,GT36864,X,Y,10,12,15,17,17.3,20,11,11,11,12,14,19,13,14,16,17.3,31.2,32.2,11,16,19,25,10,14,15,16,9,12,11,19,8,12,13,13,5,7,6,9,12,12,19,21,10,11,22.2,28.2,7,9.3,9,11,16,18,14,12,28,23,10,11,13,16,10,11,20,14,15,24,11,13,20,16,21,12,11
Cauc,GT36866,X,Y,11,11,13,16,18,20,11,12,11,13,13,15,12,14,11,14,31.2,32.2,16,17,18,20,11,14,16,17,11,13,10,18,10,10,10,10,6,6,6,10,11,11,21,21,11,11,25.2,30.2,6,9.3,8,8,16,17,14,13,29,23,10,11,12,15,9,11,21,15,14,23,12,13,16,17,23,11,11
Cauc,GT36875,X,Y,11,11,13,14,18,19,11,12,11,13,14,17,12,14,11,18.3,28,28,15,16,24,25,11.3,14,14,16,12,13,11,12,10,11,12,13,5,6,9,10,10,12,20,24,10,11,14,16,8,9.3,8,8,16,19,14,13,29,23,10,13,13,15,12,11,19,16,17,22,12,12,17,20,23,10,12
Cauc,GT36877,X,Y,11,11,14,15,17,20,11,11,9,11,16,18,13,16.2,12,15.3,28,30,11,16,18,22,11,12,15,18,11,13,12,14,9,9,12,15,6,7,8,10,10,11,20,26,10,11,15,19,6,8,8,11,15,17,15,12,30,21,10,11,14,16,10,12,22,17,16,22,10,12,17,15,21,13,11
Cauc,GT36878,X,Y,10,12,13,14,18,18,11,13,8,11,13,17,13,15,15,17.3,30,32.2,14,15,17,18,11,14,15,18,9,11,11,17,9,12,11,13,6,6,8,8,10,11,19,23,12,12,19,28.2,9.3,9.3,8,11,16,19,14,13,29,24,11,13,13,15,12,12,19,16,18,22,13,15,17,18,23,10,11
Cauc,GT36884,X,Y,10,11,13,13,17,18,11,11,8,12,14,15,12,15,12,13,28,30,16,16,17,24,13,14,14,16,11,12,12,13,8,10,11,11,5,7,6,8,11,11,23,24,12,13,17,19,7,9.3,8,11,17,18,14,13,29,24,11,13,13,15,12,12,19,16,16,22,14,11,17,20,24,10,12


##### Question 1
Look at this table more closely. Is there anything you notice about the XXXX versus XXXX.1 columns that suggest a problem with ignoring the latter, which has the second allele?  To help understand this, write code to compare histograms of the two columns associated with a few of the STR loci).

In [ ]:
# Question 1 - code

##### Question 1 - text

What's wrong with ignoring the ".1" allele?  

*Give your textual answer here*



### Restricting to original CODIS core loci but keeping information in both columns
Just like last time, we will restrict to the original CODIS loci.   We'll also keep 'Group' because it might be useful for further analysis later.

However, unlike last time, we'll include the information from both XXXX and XXXX.1 columns. There is more than one way to do this, so feel free to do it your way.

##### Question 2
Create a new table, CODIS_profiles_new, with the CODIS loci and *'Group'*, but this time combine the allele values from both allele columns in the original. For example the 'TPOX' column in CODIC_profiles_new will contain the values in both 'TPOX' and 'TPOX.1' in the old CODIS_profiles.

There are many ways to do it.  Please code this yourself if you can, but otherwise, you can use the code below... except you need to replace the '???' in the antepenultimate line.


In [ ]:
# Question 2
# Append a copy of the 'Group' column
# to the original column
group=raw_profiles.column('Group')
new_group=np.array(list(group)*2)
# when you multiply a list by k, it concatenates
# itself with itself k-1 times

# initialize a new table with the column new_group
CODIS_profiles_new=Table()
CODIS_profiles_new.append_column('Ethnicity', new_group)

# for the other columns, loop through the loci, concatenate
# XXXX and XXXX.1 columns, and append to table
CODIS_loci=['TPOX', 'D3S1358', 'FGA', 'D5S818',
            'CSF1PO', 'D7S820', 'D8S1179', 'TH01', 'vWA',
            'D13S317', 'D16S539', 'D18S51', 'D21S11']

for l in CODIS_loci:
    col1=raw_profiles.column(l)
    col2=raw_profiles.column(l+'???') # concatenate strings to get name of second column
    new_col=np.array(list(col1)+list(col2)) # concatenate lists, make into array
    CODIS_profiles_new.append_column(l, new_col)

CODIS_profiles_new

ValueError: ignored

##### Question 3
Check that the distributions of alleles you get from CODIS_profiles_new do reflect the data from the second columns for each STR locus, by plotting a histogram of the new columns.



In [ ]:
# Question 3

### Let's Calculate Match Probabilities

In [ ]:
CODIS_profiles_new.group('TPOX')

ValueError: ignored

Allele 8 is the most common allele for TPOX. Last time, to calculate an upper bound on the match probability, we assumed that an individual's DNA fingerprint consists of one allele per STR locus. i.e., their TPOX allele was 8.  But few human cells are haploid.

##### Question 4
This time, have each DNA fingerprint take account of two alleles for each STR locus, and assume that each allele is the most common one for each locus, e.g., the TPOX allele would be (8,8).

Calculate the probability that an individual has the most common alleles (on both chromosomes) for each of the 13 STR loci?

In [ ]:
#Question 4

##### Question 5
Generate two diploid DNA fingerprints arbitrarily. Do this any way you want, so long as alleles in the fingerprint are present within CODIS_profiles_new.

In [ ]:
# Question 5

##### Question 6
Probably those two fingerprints are not identical.  Calculate the probability that these two would have been identical at each diploid locus -- and then across all 13 diploid loci

##### Question 7

In the extra section, we'll see how the Ethnicity can impact matches.

Mention a few ways in which people of different backgrounds might be unequally likely to identified and found guilty using matches to CODIS data

##### Answer Question 7 Here

### Extra (optional):

##### Question 8
Use Bayes Rule and the size of the US population to etsimate the probability of innocence given a match between to profiles.

In [ ]:
# Question 8


##### Question 9
Suppose we also know the ethnicity of an individual. What is the probability that their DNA fingerprint is the most common for their ethnic group? For this, you'll need to use the 'Ethnicity' (originally labeled 'Group') column and pivot, pivot_hist, functions.


In [ ]:
# Question 9

Congrats on finishing Lab 3b!

Adapted from Shishi Luo by Sarp Dora Kurtoglu and Steven E. Brenner for UC Berkeley PMB/MCB/BioE c146. September 2022.